In [1]:
import onnx
import tvm
from tvm import relay
from tvm.contrib import graph_executor

# Load the ONNX model
onnx_model = onnx.load("model.onnx")

# Correct input name and shape based on the model's input specification
input_name = "input.1"  # Update the input name
input_shape = {"input.1": (1, 1, 28, 28)}  # Update the input shape if necessary

# Convert the ONNX model to a Relay module
mod, params = relay.frontend.from_onnx(onnx_model, input_shape)

# Set the target (e.g., "llvm" for CPU, "cuda" for GPU)
target = "cuda"  # Change to "cuda" if you have a GPU

# Build the model
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

# Create a TVM runtime executor
dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

# Prepare input data (example: random input)
import numpy as np
input_data = np.random.uniform(size=input_shape[input_name]).astype("float32")
module.set_input(input_name, input_data)

# Run the model
module.run()

# Get the output
output = module.get_output(0)
print(output)


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


[[-169.81454  -149.78537   -49.99475   -80.75202   -64.88769   -94.23434
   -87.808624  -93.77419  -146.7838    -51.877163]]


In [2]:
print(onnx_model.graph.input)


[name: "input.1"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 28
      }
      dim {
        dim_value: 28
      }
    }
  }
}
]
